In [3]:
import os
os.chdir(r"C:\Users\hp\Documents\ML_Projects\Delhi_Infrastructure_Risk_AI")
print(os.getcwd())


C:\Users\hp\Documents\ML_Projects\Delhi_Infrastructure_Risk_AI


In [4]:
import pandas as pd
import numpy as np


In [5]:
risk_df = pd.read_csv("data/processed/risk_predictions.csv")
flood_df = pd.read_csv("data/processed/flood_impact_predictions.csv")


In [6]:
risk_df.shape, flood_df.shape


((5000, 10), (5000, 9))

In [7]:
df = risk_df.merge(
    flood_df[["asset_id", "predicted_flood_impact"]],
    on="asset_id",
    how="left"
)

df.head()


,asset_id,is_flyover,SAI,TSI,FEI,MNS,CDI,risk_score,risk_label,predicted_risk,predicted_flood_impact
0,0,0,0.462255,0.435818,0.723722,0.509964,0.298624,0.519987,Medium,Medium,0.728511
1,1,1,0.303309,0.683946,0.515795,0.417737,0.127659,0.459971,Medium,Medium,0.625080
2,2,0,0.439925,0.561566,0.225722,0.495537,0.350034,0.420632,Medium,Medium,0.468781
3,3,0,0.228001,0.413710,0.401589,0.260607,0.334443,0.328038,Low,Low,0.460757
4,4,0,0.412274,0.333565,0.554517,0.359272,0.379232,0.418555,Medium,Medium,0.588250


In [8]:
def normalize(col):
    return (col - col.min()) / (col.max() - col.min())

df["risk_score_norm"] = normalize(df["risk_score"])
df["flood_impact_norm"] = normalize(df["predicted_flood_impact"])


In [9]:
df["D_CIPI"] = (
    0.40 * df["risk_score_norm"] +
    0.30 * df["flood_impact_norm"] +
    0.20 * df["MNS"] +
    0.10 * df["TSI"]
)


In [10]:
df["priority_level"] = pd.cut(
    df["D_CIPI"],
    bins=[0, 0.4, 0.7, 1.0],
    labels=["Low", "Medium", "High"]
)

df["priority_level"].value_counts()


priority_level
Medium    2781
Low       2109
High       110
Name: count, dtype: int64

In [11]:
df = df.sort_values(by="D_CIPI", ascending=False)

df[["asset_id", "D_CIPI", "priority_level"]].head(10)


,asset_id,D_CIPI,priority_level
1303,1303,0.898254,High
1418,1418,0.854068,High
3557,3557,0.837911,High
2211,2211,0.834831,High
1265,1265,0.832058,High
3462,3462,0.829035,High
1795,1795,0.825781,High
2352,2352,0.817009,High
4075,4075,0.816906,High
4595,4595,0.809107,High


In [12]:
df.to_csv(
    "data/processed/infrastructure_priority_index.csv",
    index=False
)
